In [13]:
import re
import codecs
import jieba
import pickle
import pandas as pd
from elasticsearch import Elasticsearch

In [14]:
es = Elasticsearch([{'host':'localhost','port':9200}])

In [15]:
# all_question = pd.read_csv('./data/description.csv')
all_question = pd.read_excel('./data/Herceptin.xlsx')
# all_question = pd.read_excel('./data/Herceptin_funding_filtered.xlsx')
# all_question = pd.read_excel('./data/Herceptin_patient_filtered.xlsx')

In [16]:
all_question.dropna(axis=0, inplace=True, subset=['Description'])

In [17]:
all_question.isna().sum()

Case Number    0
Description    0
dtype: int64

In [18]:
jieba.load_userdict('./chinese_word_dict.txt')
chinese_word_dict = pickle.load(open('./chinese_word_dict.pickle', 'rb'))
stopwords = [line.strip() for line in codecs.open('./stopwords-zh.txt', 'r', encoding='utf8').readlines()]

In [19]:
def keyword_extract(text, word_dict):
#     text = list(jieba.cut(text, cut_all=False))
    text = list(jieba.cut_for_search(text))
    union_text = list(set(text).intersection(set(word_dict)))
    return " ".join(union_text)

In [20]:
all_question['KeyWords'] = all_question.Description.apply(lambda x: keyword_extract(x, chinese_word_dict))

In [21]:
# 去除所有“[]”内的文本
def remove_special_string(text):
    text = re.sub(r'\[[^]]*\]', '', text)
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub(r'（[^）]*\）', '', text)
    text = re.sub(r'\<[^>]*\>', '', text)
    text = re.sub(r'[0-9]*', '', text)
    return text
def remove_stopwords(text, stopwords):
    text_cutted = jieba.cut(text, cut_all=True)
    text_removed = []
    for word in text_cutted:
        if word in stopwords or word.strip()=="":
            continue
        text_removed.append(word)
    return "  ".join(text_removed)

In [22]:
all_question['Processed'] = all_question.Description.apply(lambda x: remove_special_string(x))
all_question['Processed'] = all_question.Processed.apply(lambda x: remove_stopwords(x, stopwords))
all_question.head()

,Case Number,Description,KeyWords,Processed
0,1420656.0,"赫赛汀 （批号1： N3825B03B3049, 批号2： N3826B02B3050） 配...",赫赛汀 使用,配 之后 结冰 继续 使用
1,1423956.0,赫赛汀粉末形状如何？患者保存的赫赛汀的稀释液出现结冰情况，是否能否使用（无产品批号信息）,无 赫赛汀 患者 使用,粉末 形状 保存 稀释 稀释液 出现 结冰 冰情 情况 是否 使用
2,1434184.0,[Wechat] 请帮忙查一些赫赛汀与血栓栓塞方面的数据和文献（确认无AE）,文献 血栓栓塞 赫赛汀 查 栓塞 血栓 无,帮忙 查 血栓 血栓栓塞 栓塞 方面 数据 和文 文献
3,1436011.0,拿到的赫赛汀呈现块状是否正常,赫赛汀 正常,拿到 呈现 块状 是否 正常
4,1438071.0,常规配置和保存的赫赛汀复溶后的溶液保存在2-8度的情况下，现出现浑浊情况，能否使用,溶液 赫赛汀 配置 现 使用 下,常规 配置 保存 复 溶 溶液 保存 存在 度 情况 现出 出现 浑...


In [23]:
doc_list = []
for idx,row in all_question.iterrows():
    doc = {"text": row['Description'], "processed_text": row['Processed']}
    doc_list.append(doc)
print(len(doc_list))
print(doc_list[0])

757
{'text': '赫赛汀 （批号1：\xa0N3825B03B3049, 批号2： N3826B02B3050） 配了之后结冰了， 能否继续使用。', 'processed_text': '配  之后  结冰  继续  使用'}


In [24]:
for idx in range(len(doc_list)):
    es.index(index='index', doc_type='type', id=idx, body=doc_list[idx])

In [29]:

query = "关于赫赛汀心脏毒性：1、临床 2、临床前"

results = es.search(index='index', body={'query': {'match':{"text": query}}}, size=10)
hits = results['hits']['hits']
results
# for idx, res in enumerate(hits):
#     print(str(idx+1) + ":  " + res['_source']['text'])

{'took': 7,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 726, 'relation': 'eq'},
  'max_score': 62.157417,
  'hits': [{'_index': 'index',
    '_type': 'type',
    '_id': '510',
    '_score': 62.157417,
    '_source': {'text': '关于赫赛汀心脏毒性：1、临床 2、临床前',
     'processed_text': '心脏  心脏毒性  脏毒  毒性  临床  临床  前'}},
   {'_index': 'index',
    '_type': 'type',
    '_id': '620',
    '_score': 38.22045,
    '_source': {'text': '[Wechat] 临床上赫赛汀q3W和qW用法，在心脏毒性方面，哪个频率心脏毒性小些？相关研究？',
     'processed_text': '临床  床上  qW  qW  用法  心脏  心脏毒性  脏毒  毒性  方面  频率  心脏  心脏毒性  脏毒  毒性  相关  研究'}},
   {'_index': 'index',
    '_type': 'type',
    '_id': '501',
    '_score': 21.81059,
    '_source': {'text': '[Wechat] 赫赛汀抑制二聚体形成的临床研究',
     'processed_text': '抑制  二聚体  体形  形成  临床  临床研究  研究'}},
   {'_index': 'index',
    '_type': 'type',
    '_id': '667',
    '_score': 21.803432,
    '_source': {'text': '[Wechat] 赫赛汀对于心脏毒性的相关文献以及处理方法',
     'processed_te